In [37]:
import logging
import pandas as pd
import os
import random
import time, datetime
import requests
import json
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.query_api import QueryApi

In [38]:
# Set up logging
logging.basicConfig(filename='met_norway_requests.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Script started at %s', pd.Timestamp.now(tz='UTC'))

In [39]:
# Define InfluxDB connection details
influxdb_url = 'http://159.89.103.242:8086'
influxdb_token = 'dfRMxqDtwyHK7vDJHelAm0WKISLvKFUrmhclvaaAoMFOHRRTGNnYkV8bXd0jR9r4arvkg3l_lWNSHyKMG0WxSg=='
influxdb_org = 'entra'
influxdb_bucket = 'entra'

In [40]:
# Create InfluxDB client
influx_client = InfluxDBClient(url=influxdb_url, token=influxdb_token, org=influxdb_org)

In [41]:
# Create the write API
write_api = influx_client.write_api(write_options=SYNCHRONOUS)

In [42]:
# Base URL for Frost API
base_url = "https://frost.met.no/observations/v0.jsonld"

In [43]:
client_id = '1ee80452-6206-498b-ae37-3c43c60dce1a'
client_secret = '674e9b24-838e-4b4c-9710-f6e001f8ce16'

In [44]:
# This should be replaced by actual coordinates
locations = {
    "country_1": (0, 0),
    "BG": (42.6977, 23.3219)
}

In [35]:
#Cities choosen for each country code
country_city = {
    "AL": "Tirana",
    "AT": "Vienna",
    "BA": "Sarajevo",
    "BE": "Brussels",
    "BG": "Sofia",
    "BY": "Minsk",
    "CH": "Bern",
    "CWE": "Manual",
    "CY": "Nicosia",
    "CZ": "Prague",
    "CZ_DE_SK": "Manual",
    "DE": "Berlin",
    "DE_50HZ": "Manual",
    "DE_AMPRION": "Manual",
    "DE_AT_LU": "Manual",
    "DE_LU": "Manual",
    "DE_TENNET": "Manual",
    "DE_TRANSNET": "Manual",
    "DK": "Copenhagen",
    "DK_1": "Manual",
    "DK_2": "Manual",
    "DK_CA": "Manual",
    "EE": "Tallinn",
    "ES": "Madrid",
    "FI": "Helsinki",
    "FR": "Paris",
    "GB": "London",
    "GB_NIR": "Belfast",
    "GR": "Athens",
    "HR": "Zagreb",
    "HU": "Budapest",
    "IE": "Dublin",
    "IE_SEM": "Manual",
    "IS": "Reykjavik",
    "IT": "Rome",
    "IT_BRNN": "Manual",
    "IT_CNOR": "Manual",
    "IT_CSUD": "Manual",
    "IT_FOGN": "Manual",
    "IT_GR": "Manual",
    "IT_MACRO_NORTH": "Manual",
    "IT_MACRO_SOUTH": "Manual",
    "IT_MALTA": "Valletta",
    "IT_NORD": "Milan",
    "IT_NORD_AT": "Manual",
    "IT_NORD_CH": "Manual",
    "IT_NORD_FR": "Manual",
    "IT_NORD_SI": "Manual",
    "IT_PRGP": "Manual",
    "IT_ROSN": "Manual",
    "IT_SACO_AC": "Manual",
    "IT_SACO_DC": "Manual",
    "IT_SARD": "Cagliari",
    "IT_SICI": "Manual",
    "IT_SUD": "Naples",
    "LT": "Vilnius",
    "LU": "Luxembourg",
    "LV": "Riga",
    "MD": "Chisinau",
    "ME": "Podgorica",
    "MK": "Skopje",
    "MT": "Valletta",
    "NL": "Amsterdam",
    "NO": "Oslo",
    "NO_1": "Manual",
    "NO_2": "Manual",
    "NO_3": "Manual",
    "NO_4": "Manual",
    "NO_5": "Manual",
    "PL": "Warsaw",
    "PL_CZ": "Manual",
    "PT": "Lisbon",
    "RO": "Bucharest",
    "RS": "Belgrade",
    "RU": "Moscow",
    "RU_KGD": "Kaliningrad",
    "SE": "Stockholm",
    "SE_1": "Manual",
    "SE_2": "Manual",
    "SE_3": "Manual",
    "SE_4": "Manual",
    "SI": "Ljubljana",
    "SK": "Bratislava",
    "TR": "Ankara",
    "UA": "Kyiv",
    "UA_BEI": "Manual",
    "UA_DOBTPP": "Manual",
    "UA_IPS": "Manual",
}

In [45]:
#Coordinates of city chosen for each country code
locations = {
    "AL": (41.3275, 19.8187),
    "AT": (48.2082, 16.3738),
    "BA": (43.8563, 18.4131),
    "BE": (50.8503, 4.3517),
    "BG": (42.6977, 23.3219),
    "BY": (53.9045, 27.5615),
    "CH": (46.9480, 7.4474),
    "CWE": "Manual",
    "CY": (35.1856, 33.3823),
    "CZ": (50.0755, 14.4378),
    "CZ_DE_SK": "Manual",
    "DE": (52.5200, 13.4050),
    "DE_50HZ": "Manual",
    "DE_AMPRION": "Manual",
    "DE_AT_LU": "Manual",
    "DE_LU": "Manual",
    "DE_TENNET": "Manual",
    "DE_TRANSNET": "Manual",
    "DK": (55.6761, 12.5683),
    "DK_1": "Manual",
    "DK_2": "Manual",
    "DK_CA": "Manual",
    "EE": (59.4370, 24.7536),
    "ES": (40.4168, -3.7038),
    "FI": (60.1756, 24.9342),
    "FR": (48.8566, 2.3522),
    "GB": (51.5074, -0.1278),
    "GB_NIR": (54.5973, -5.9301),
    "GR": (37.9838, 23.7275),
    "HR": (45.8150, 15.9819),
    "HU": (47.4979, 19.0402),
    "IE": (53.3498, -6.2603),
    "IE_SEM": "Manual",
    "IS": (64.1466, -21.9426),
    "IT": (41.9028, 12.4964),
    "IT_BRNN": "Manual",
    "IT_CNOR": "Manual",
    "IT_CSUD": "Manual",
    "IT_FOGN": "Manual",
    "IT_GR": "Manual",
    "IT_MACRO_NORTH": "Manual",
    "IT_MACRO_SOUTH": "Manual",
    "IT_MALTA": (35.8989, 14.5146),
    "IT_NORD": (45.4642, 9.1900),
    "IT_NORD_AT": "Manual",
    "IT_NORD_CH": "Manual",
    "IT_NORD_FR": "Manual",
    "IT_NORD_SI": "Manual",
    "IT_PRGP": "Manual",
    "IT_ROSN": "Manual",
    "IT_SACO_AC": "Manual",
    "IT_SACO_DC": "Manual",
    "IT_SARD": (39.2238, 9.1217),
    "IT_SICI": "Manual",
    "IT_SUD": (40.8522, 14.2681),
    "LT": (54.6872, 25.2797),
    "LU": (49.6116, 6.1319),
    "LV": (56.9496, 24.1052),
    "MD": (47.0105, 28.8638),
    "ME": (42.4304, 19.2594),
    "MK": (41.9973, 21.4280),
    "MT": (35.8989, 14.5146),
    "NL": (52.3676, 4.9041),
    "NO": (59.9139, 10.7522),
    "NO_1": "Manual",
    "NO_2": "Manual",
    "NO_3": "Manual",
    "NO_4": "Manual",
    "NO_5": "Manual",
    "PL": (52.2297, 21.0122),
    "PL_CZ": "Manual",
    "PT": (38.7223, -9.1393),
    "RO": (44.4268, 26.1025),
    "RS": (44.7866, 20.4489),
    "RU": (55.7558, 37.6176),
    "RU_KGD": (54.7101, 20.5110),
    "SE": (59.3293, 18.0686),
    "SE_1": "Manual",
    "SE_2": "Manual",
    "SE_3": "Manual",
    "SE_4": "Manual",
    "SI": (46.0569, 14.5058),
    "SK": (48.1486, 17.1077),
    "TR": (39.9334, 32.8597),
    "UA": (50.4501, 30.5234),
    "UA_BEI": "Manual",
    "UA_DOBTPP": "Manual",
    "UA_IPS": "Manual",
}

In [47]:
for country, coords in locations.items():
    try:
        # Define the parameters for the API request
        parameters = {
            'sources': 'SN18700',  # replace with your actual station identifier
            'elements': 'air_temperature,ultraviolet_index',
            'referencetime': '2023-01-01T00:00:00Z/2023-07-18T00:00:00Z'
        }

        # Make the API request
        response = requests.get(base_url, parameters, auth=(client_id, ''))

        # Raise an error if the request was unsuccessful
        response.raise_for_status()

        # Parse the response
        data = json.loads(response.text)
        print(f"Request completed for {country}.")
        print(data)  # print the data
        
        # Process the data and write to InfluxDB
        for entry in data["data"]:
            timestamp = entry["referenceTime"]
            for observation in entry["observations"]:
                element = observation["elementId"]
                value = observation["value"]

                # Create data point
                data_point = Point(element) \
                    .tag("country", country) \
                    .field("value", float(value)) \
                    .time(timestamp, WritePrecision.NS)

                # Write the data point to InfluxDB
                write_api.write(bucket=influxdb_bucket, record=data_point)
                
        logging.info(f'Data extraction completed for country: {country}')
    except Exception as e:
        logging.error(f'Data extraction failed for country: {country} : {str(e)}')

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)


KeyboardInterrupt



In [ ]:
for country, coords in locations.items():
    try:
        # Define the parameters for the API request
        parameters = {
            'sources': 'SN18700',  # replace with your actual station identifier
            'elements': 'ultraviolet_index',
            'referencetime': '2022-01-01T00:00:00Z/2023-07-18T00:00:00Z'
        }

        # Make the API request
        response = requests.get(base_url, parameters, auth=(client_id, ''))

        # Raise an error if the request was unsuccessful
        response.raise_for_status()

        # Parse the response
        data = json.loads(response.text)
        print(f"Request completed for {country}.")
        print(data)  # print the data
        
        # Process the data and write to InfluxDB
        for entry in data["data"]:
            timestamp = entry["referenceTime"]
            for observation in entry["observations"]:
                element = observation["elementId"]
                value = observation["value"]

                # Create data point
                data_point = Point(element) \
                    .tag("country", country) \
                    .field("value", float(value)) \
                    .time(timestamp, WritePrecision.NS)

                # Write the data point to InfluxDB
                write_api.write(bucket=influxdb_bucket, record=data_point)
                
        logging.info(f'Data extraction completed for country: {country}')
    except Exception as e:
        logging.error(f'Data extraction failed for country: {country}, source: {source}: {str(e)}')

In [50]:
#Get a list of all Station Identifiers
base_url = 'https://frost.met.no/sources/v0.jsonld'

response = requests.get(base_url, auth=(client_id, ''))

# Raise an error if the request was unsuccessful
response.raise_for_status()

# Parse the response
stations = json.loads(response.text)

# Loop through the stations and print station identifiers and names
for item in stations['data']:
    if 'id' in item and 'name' in item:
        print(f"Station ID: {item['id']}, Name: {item['name']}")
    else:
        print(f"One or both keys are missing in the item: {item}")


Station ID: SN47230, Name: ÅKRA UNGDOMSSKOLE
Station ID: SN23670, Name: E16 RYFOSS
Station ID: SN59450, Name: STADLANDET
Station ID: SN12590, Name: E6 MJØSBRUA
Station ID: SN26640, Name: E134 DARBU
Station ID: SN67650, Name: FV30 REITSTØA
Station ID: SN18269, Name: OSLO - HAUGENSTUA
Station ID: SN57790, Name: MYKLEBUSTFJELLET
Station ID: SN99895, Name: KVADEHUKEN II
Station ID: SN233100, Name: HUNGE
Station ID: SN61065, Name: E136 TRESFJORDBRUA
Station ID: SN1039300, Name: LINDENBERG
Station ID: SN743400, Name: LIMOGES-BELLEGARDE
Station ID: SN27600, Name: SANDEFJORD
Station ID: SN18700, Name: OSLO - BLINDERN
Station ID: SN87110, Name: ANDØYA
Station ID: SN40905, Name: RV9 HOVDEN
Station ID: SN27295, Name: FV307 GRAVDAL
Station ID: SN40250, Name: VALLE
Station ID: SN224720, Name: FORSE
Station ID: SN43220, Name: FV44 SÆLEBØ
Station ID: SN19660, Name: E16 SKUI
Station ID: SN18260, Name: OSLO - HØYBRÅTEN
Station ID: SN49240, Name: RV13 LÅTEFOSS 
Station ID: SN1240000, Name: ZIELONA GORA
